In [2]:
import ccxt
import joblib
import time
import numpy as np
import pandas as pd
import sys
from keras.models import load_model
from datetime import datetime
sys.path.append("/Users/yuhyeonseog/COBOK/COBOK/convergence/indicator")
from DATA_INDICATORS import *
def download_data(frame):
  binance = ccxt.binance()
  btc_ohlcv = binance.fetch_ohlcv("BTC/USDT",limit = 500,timeframe=frame)
  df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
  df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
  df.set_index('datetime', inplace=True)
  return df

model = load_model("../model/keras_DNN_modelV2")
trade_history = []
api_key = "598lqCWX727NXfflVt1qejRSZXphSuWVlcdMm1gdtAVD3lytdDhjpbV3eyxcr0z1"
secret  = "XxDx9DuVDiYngMV71jWHHCFYRhhph4bdZdGCSH4JVb2s2jCJtHW6gbpPwZWjfYvf"

binance = ccxt.binance(config={
    'apiKey': api_key, 
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
# # 레버리지 변경
# markets = binance.load_markets()
# symbol = "BTC/USDT"
# market = binance.market(symbol)
# leverage = 10

# resp = binance.fapiPrivate_post_leverage({
#     'symbol': market['id'],
#     'leverage': leverage
# })

2023-01-19 21:00:08.237574: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [77]:
leverage = 10
markets = binance.load_markets()
symbol = "BTC/USDT"
market = binance.market(symbol)
resp = binance.fapiPrivate_post_leverage({
    'symbol': market['id'],
    'leverage': leverage
})
def buy(Account,price):
  Account["average_price"] = ( ( Account["amount"] * Account["average_price"] ) + ( price * 0.01 ) ) / ( Account["amount"] + 0.01 )
  Account["amount"] += 0.01
  return Account

def sell(Account,price):
  Account["result"] += (price - Account["average_price"]) * Account["amount"]
  Account["amount"] = 0
  Account["average_price"] = 0
  return Account

def info():
  balance = binance.fetch_balance(params={"type": "future"})
  Now_price = binance.fetch_ticker("BTC/USDT")["close"]
  positions = balance['info']['positions']

  for position in positions:
      if position["symbol"] == "BTCUSDT":
          Now_price = float(binance.fetch_ticker("BTC/USDT")["close"])
          entry_price = float(position["entryPrice"])
          leverage = float(position["leverage"])
          Margin = float(position["positionInitialMargin"])
          ROE = round((Now_price - entry_price) / Now_price * leverage * 100 ,2)
          print("현재 코인 가격 :",Now_price)
          print("현재 보유 수량 :",position["positionAmt"]) # 수량
          print("현재 평단가 :",position["entryPrice"]) # 진입가격 -> 평단가       
          print("현재 적용 레버리지 :",position["leverage"]) # 레버리지
          if position["positionAmt"] == 0:
            print("현재 포지션 수익률 :",0.0,"%")
          else:
            print("현재 포지션 수익률 :",ROE,"%")
          print("현재 포지션 수익(달러) :",round(ROE/100 * Margin,2),"$")
def Re_ROE():
  balance = binance.fetch_balance(params={"type": "future"})
  Now_price = binance.fetch_ticker("BTC/USDT")["close"]
  positions = balance['info']['positions']
  for position in positions:
      if position["symbol"] == "BTCUSDT":
          Now_price = float(binance.fetch_ticker("BTC/USDT")["close"])
          entry_price = float(position["entryPrice"])
          leverage = float(position["leverage"])
          ROE = round((Now_price - entry_price) / Now_price * leverage * 100 ,2)
          break
  return ROE

def Re_Amount():
  balance = binance.fetch_balance(params={"type": "future"})
  Now_price = binance.fetch_ticker("BTC/USDT")["close"]
  positions = balance['info']['positions']
  amount = 0
  for position in positions:
      if position["symbol"] == "BTCUSDT":
          amount = position["positionAmt"]
          break
  return amount
def trade():
  Account = {
    "result" : 0,
    "amount" : 0,
    "average_price" : 0
  }
  previous_time = '2018-01-01 00:00:00'
  buy_min = -1
  while(True):
    data = download_data('1m')
    df1 = data
    df1 = add_rsi(df1)
    df1 = add_ma(df1,period=7)
    df1 = add_ema(df1,period=7)
    df1 = add_ma(df1,period=25)
    df1 = add_ema(df1,period=25)
    df1 = add_ma(df1,period=99)
    df1 = add_ema(df1,period=99)
    df1 = add_stochastic(df1)
    df1 = add_bb(df1,length=21)
    df1 = add_disparity(df1,period=25)
    df1 = add_macd(df1)
    df1 = add_kdj(df1)
    df1 = df1.dropna()


    x = df1.iloc[-10:]
    pred = model.predict(x)

    Trading_Flag = -1

    if pred[-1][0] > 0.92:
      Trading_Flag = 1
    elif pred[-1][0] < 0.1:
      Trading_Flag = 0

    now = datetime.now()
    now_min = now.minute

    # 이전과 시간이 다르므로 조건 만족시 구매 
    if buy_min != now_min and Trading_Flag == 1:
      # order = binance.create_market_buy_order(
      # symbol="BTC/USDT",
      # amount=0.001)
      Account = buy(Account,df1.iloc[-1]['open'])
      buy_min = now_min

    elif Trading_Flag == 0 and Account["amount"] >= 0.01 and 0.15 * leverage <= Re_ROE():
      # order = binance.create_market_sell_order(
      # symbol="BTC/USDT",
      # amount=float(Re_Amount()) )
      Account = sell(Account,df1.iloc[-1]['open'])


    print("==============================================")
    print("현재 time:",now.strftime('%Y-%m-%d %H:%M:%S'))
    info()
    print("매수매도 시그널(1:매수, 0:매도, -1:관망) :",Trading_Flag)
    print("토탈 수익:",Account["result"])
    time.sleep(5)
trade()

현재 time: 2023-01-05 04:47:00
현재 코인 가격 : 16835.2
현재 보유 수량 : 0.000
현재 평단가 : 0.0
현재 적용 레버리지 : 10
현재 포지션 수익률 : 1000.0 %
현재 포지션 수익(달러) : 0.0 $
매수매도 시그널(1:매수, 0:매도, -1:관망) : -1
토탈 수익: 0
[0.29809093]
현재 time: 2023-01-05 04:47:07


KeyboardInterrupt: 

In [ ]:
balance = binance.fetch_balance(params={"type": "future"})
print(balance['USDT'])

In [ ]:
binance.fetch_ticker("BTC/USDT")

In [ ]:
binance.fetch_ticker("BTC/USDT")["close"]

In [ ]:
import pprint
# balance = binance.fetch_balance()
positions = balance['info']['positions']

In [ ]:
for position in positions:
    if position["symbol"] == "BTCUSDT":
        Now_price = float(binance.fetch_ticker("BTC/USDT")["close"])
        entry_price = float(position["entryPrice"])
        leverage = float(position["leverage"])
        Margin = float(position["positionInitialMargin"])
        ROE = round((Now_price - entry_price) / Now_price * leverage * 100 ,2)
        print(position["positionAmt"]) # 수량
        print(position["entryPrice"]) # 진입가격 -> 평단가
        print(position["leverage"]) # 레버리지
        print(ROE,"%")
        print(round(ROE/100 * Margin,2),"$")
        pprint.pprint(position)

In [ ]:
# 수량
positions

In [72]:
from datetime import datetime

now = datetime.now()
now.minute

33

In [1]:
import pandas as pd

In [5]:
data = pd.read_csv("Data/TradingData.csv").drop(['Unnamed: 0'],axis = 1)
data

,raw_Time,time,quantity,average,leverage,profit_rate,profit,signal,total_profit
0,1674126878000,2023-01-19 20:14:38,0.0,0.0,0,0.0,0.0,-1,0


In [13]:
result = {}
for i in data.columns:
    result[i] = str(data[i].iloc[0])
result

{'raw_Time': '1674126878000',
 'time': '2023-01-19 20:14:38',
 'quantity': '0.0',
 'average': '0.0',
 'leverage': '0',
 'profit_rate': '0.0',
 'profit': '0.0',
 'signal': '-1',
 'total_profit': '0'}

In [14]:
import json
json.dumps(result)

'{"raw_Time": "1674126878000", "time": "2023-01-19 20:14:38", "quantity": "0.0", "average": "0.0", "leverage": "0", "profit_rate": "0.0", "profit": "0.0", "signal": "-1", "total_profit": "0"}'

In [7]:
# binance 객체 생성
binance = ccxt.binance(config={
    'apiKey': "598lqCWX727NXfflVt1qejRSZXphSuWVlcdMm1gdtAVD3lytdDhjpbV3eyxcr0z1",
    'secret': "XxDx9DuVDiYngMV71jWHHCFYRhhph4bdZdGCSH4JVb2s2jCJtHW6gbpPwZWjfYvf",
})

# USDT의 잔고 조회
balance = binance.fetch_balance(params={"type": "future"})
print(balance['USDT'])

{'free': 111.2770928, 'used': 2.07249, 'total': 113.34069936}


In [8]:
balance['USDT']['used']

2.07249